# Segmenting and clustering neighbourhood in Toronto

### Scraping and Data wrangling

In [61]:
#installing the necessary libraries
!pip install BeautifulSoup4
!pip install requests
!pip install lxml
!pip install folium
!pip install geopy
#importing the libararies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import lxml
import io
import matplotlib.pyplot as plt
import folium
import json
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### The wikipedia page is accessed using beautiful soup, requests and lxml libraries. A dictionary is created by making the keys as the column headers whose values are an empty list initially. Later on, the lists are appended in a for loop using necessary html tags.

In [ ]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(page,'lxml')

table = soup.find('table', class_ = 'wikitable sortable')

table_dict ={'Postcode':[],'Borough':[],'Neighbourhood':[]}

for tr in table.find_all('tr'):
    tds = tr.find_all_next('td')
    table_dict['Postcode'].append(tds[0].text)
    table_dict['Borough'].append(tds[1].text)
    table_dict['Neighbourhood'].append(tds[2].text)
    
table_dict['Neighbourhood'] = [i.replace('\n','') for i in table_dict['Neighbourhood']]

#### The dictionary previously created is converted into a dataframe using pandas library and required cleaning, grouping and replacements are done.

In [8]:
#Creating the dataframe
df = pd.DataFrame(table_dict)

#eliminating the rows with 'Not assigned' borough
df = df[df.Borough != 'Not assigned']
#grouping
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

df.shape

(103, 3)

In [9]:
df.head(50)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Including the Latitude and Lognitude to the dataframe

In [12]:
# in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postcode')

# reorder column names and show the dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


### Using the geopy library to obtain the latitudes and longitudes of toronto

In [195]:
df = df[['Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")# define the user agent as toronto explorer
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Create a map of toronto with the neighbourhood superimposed on top.

In [31]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [213]:
toronto_df = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)


toronto_df = toronto_df[:-1]
print(toronto_df.shape)

(38, 4)


### Visiting toronto and its neighbourhood only.

In [214]:
# create map of New York using latitude and longitude values
map_toronto_n = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_n)  
    
map_toronto_n

### Define Foursquare Credentials and Venues

In [215]:
CLIENT_ID = 'XNV4WEQAV43Y2UJ0D34LOG5FTG4LBMVB2YE1C1PRPALCWSTC' # your Foursquare ID
CLIENT_SECRET = 'FMVKOA4LN0OYA5XWXTGKWU2JWP52L1BBY0QYD0OVVOE4AD2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XNV4WEQAV43Y2UJ0D34LOG5FTG4LBMVB2YE1C1PRPALCWSTC
CLIENT_SECRET:FMVKOA4LN0OYA5XWXTGKWU2JWP52L1BBY0QYD0OVVOE4AD2J


### Exploring the first neighbourhood in our dataframe

In [216]:
toronto_df.loc[0,'Neighbourhood']

'The Beaches'

### Getting neighbourhood's latitude and longitude values

In [217]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.


#### Creating the url

In [218]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XNV4WEQAV43Y2UJ0D34LOG5FTG4LBMVB2YE1C1PRPALCWSTC&client_secret=FMVKOA4LN0OYA5XWXTGKWU2JWP52L1BBY0QYD0OVVOE4AD2J&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

#### Send the GET request and examine the results

In [219]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e03a9aaf7706a001be8db76'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [220]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Cleaning json and structure it into pandas dataframe

In [221]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


## Explore neighbourhoods in Toronto

In [223]:
#function to repeat the same process to all neighbourhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [224]:

toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

### Checking the size of the resulting dataframe

In [225]:
print(toronto_venues.shape)
toronto_venues.head()

(1682, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


##### checking how many venues were returned for each neighbourhood

In [226]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,18,18,18,18,18,18


#### Let's find out how many unique categories can be curated from all the returned venues

In [227]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyze each neighbourhood

In [228]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### New dataframe size

In [229]:
toronto_onehot.shape

(1682, 237)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [230]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.000000,0.00000,0.00000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.042553,0.000000,0.053191,0.010638,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.023529,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.011765,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.011765,0.000000,0.011765


#### Let's confirm the new size

In [231]:
toronto_grouped.shape

(38, 237)

#### Let's print each neighborhood along with the top 5 most common venues

In [232]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2          Steakhouse  0.04
3   French Restaurant  0.04
4  Seafood Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.14
1     Coffee Shop  0.10
2  Breakfast Spot  0.10
3   Burrito Place  0.05
4   Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Garden Center  0.05
3                Park  0.05
4          Comic Shop  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12

### Putting it into the pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [233]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [257]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(38, 11)

## Cluster Neighbourhoods

##### Run kmeans to cluster the neighbourhood into 10 clusters

In [258]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10]) 
print(len(kmeans.labels_))


[0 0 0 0 0 0 0 0 0 0]
38


In [251]:
toronto_df.head()
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [259]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
 
neighborhoods_venues_sorted['Cluster Labels'].astype(int)
toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head(30) # check the last columns!
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged['Cluster Labels'].astype(int)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    2
18    0
19    0
20    0
21    0
22    1
23    0
24    0
25    0
26    0
27    4
28    3
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
Name: Cluster Labels, dtype: int32

#### Finally, let's visualize the resulting clusters

In [260]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [261]:
toronto_merged

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Coffee Shop,Trail,Wings Joint,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Italian Restaurant,Bookstore,Furniture / Home Store,Indian Restaurant,Fruit & Vegetable Store,Juice Bar
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Pet Store,Pub,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Gastropub,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Seafood Restaurant,Sandwich Place
4,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Gym / Fitness Center,Park,Bus Line,Swim School,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Breakfast Spot,Food & Drink Shop,Hotel,Clothing Store,Sandwich Place,Convenience Store,Park,Ethiopian Restaurant,Discount Store
6,Central Toronto,North Toronto West,43.715383,-79.405678,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Restaurant,Rental Car Location,Miscellaneous Shop,Bagel Shop,Mexican Restaurant,Dessert Shop
7,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Gourmet Shop,Asian Restaurant
8,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Restaurant,Playground,Summer Camp,Trail,Tennis Court,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
9,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Pub,Coffee Shop,Bagel Shop,Vietnamese Restaurant,Liquor Store,Supermarket,Restaurant,Pizza Place,Sushi Restaurant,Light Rail Station


In [163]:
kmeans.labels_


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [133]:
len(kmeans.labels_)

38

In [239]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Salad Place,Asian Restaurant,Restaurant,Burger Joint,Bakery,Thai Restaurant
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Seafood Restaurant,French Restaurant,Beer Bar,Steakhouse,Bakery
2,0,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Convenience Store,Burrito Place,Stadium,Italian Restaurant,Bar,Restaurant
3,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry
5,0,"Cabbagetown, St. James Town",Pizza Place,Coffee Shop,Café,Restaurant,Bakery,Pub,Italian Restaurant,Chinese Restaurant,Playground,Plaza
6,0,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Japanese Restaurant,Bubble Tea Shop,Gym / Fitness Center,Fried Chicken Joint
7,0,"Chinatown, Grange Park, Kensington Market",Café,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bar,Mexican Restaurant,Bakery,Tea Room
8,0,Christie,Grocery Store,Café,Park,Convenience Store,Athletics & Sports,Italian Restaurant,Bank,Candy Store,Baby Store,Diner
9,0,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
